# DB 연결

In [1]:
# pip install sqlalchemy pymysql

In [2]:
import sqlalchemy
import json

with open("./db_config.json", "r") as f:
    config = json.load(f)

USER = config["DB_USER"]
PW = config["DB_PASSWORD"]
HOST = config["DB_HOST"]
PORT = config["DB_PORT"]
DB = config["DB_NAME"]

# DB 연결
engine = sqlalchemy.create_engine(f"mysql+pymysql://{USER}:{PW}@{HOST}:{PORT}/{DB}")

with engine.connect() as conn:
    result = conn.execute(sqlalchemy.text("SELECT VERSION();"))
    print("Connect Success. Version is", result.fetchone())


Connect Success. Version is ('8.0.16',)


# 1. stadium 테이블

In [3]:
# stadium_code 설정
stadium_code_map = {
    "안산상록수체육관": "AS01",
    "안양 정관장 아레나": "AY01",
    "부산 아시아드 경기장": "BS01",
    "사직야구장": "BS02",
    "부산사직체육관": "BS03",
    "천안 종합 운동장": "CA01",
    "천안유관순체육관": "CA02",
    "춘천 송암 스포츠타운": "CC01",
    "청주야구장": "CJ01",
    "창원NC파크": "CW01",
    "창원체육관": "CW02",
    "대구iM뱅크PARK": "DG01",
    "대구 스타디움": "DG02",
    "대구 삼성 라이온즈 파크": "DG03",
    "대구체육관": "DG04",
    "대전 월드컵 경기장": "DJ01",
    "대전 한화생명 볼파크": "DJ02",
    "한화생명 이글스 파크": "DJ03",
    "대전충무체육관": "DJ04",
    "김천 종합 운동장": "GC01",
    "김천실내체육관": "GC02",
    "광주 축구 전용경기장": "GJ01",
    "광주-기아 챔피언스필드": "GJ02",
    "구미박정희체육관": "GM01",
    "강릉 종합 운동장": "GN01",
    "김포솔터축구장": "GP01",
    "군산월명체육관": "GS01",
    "고양 소노 아레나": "GY01",
    "페퍼스타디움": "HS01",
    "화성종합실내체육관": "HS02",
    "인천 축구 전용경기장": "IC01",
    "인천 SSG 랜더스 필드": "IC02",
    "인천계양체육관": "IC03",
    "인천삼산월드체육관": "IC04",
    "두산베어스파크": "IH01",
    "제주 월드컵 경기장": "JE01",
    "전주 월드컵 경기장": "JJ01",
    "전주실내체육관": "JJ02",
    "포항 스틸야드": "PH01",
    "포항야구장": "PH02",
    "서울 월드컵 경기장": "SO01",
    "상암 보조경기장": "SO02",
    "목동 종합 운동장": "SO03",
    "고척 스카이돔": "SO04",
    "잠실야구장": "SO05",
    "잠실실내체육관": "SO06",
    "잠실학생체육관": "SO07",
    "서울장충체육관": "SO08",
    "수원 월드컵 경기장": "SU01",
    "수원 종합 운동장": "SU02",
    "수원 케이티위즈 파크": "SU03",
    "수원 KT 아레나": "SU04",
    "수원체육관": "SU05",
    "의정부체육관": "UJ01",
    "경민대학교 기념관(체육관)": "UJ02",
    "울산 문수 축구경기장": "US01",
    "울산 종합 운동장": "US02",
    "울산문수야구장": "US03",
    "울산동천체육관": "US04",
    "원주종합체육관": "WJ01"
}

In [4]:
import pandas as pd
from datetime import datetime, timedelta

# 데이터셋 가져오기
df_stadium = pd.read_excel("./dataset/스포츠경기장명 및 주소.xlsx")

# 컬럼명 변경
df_stadium.rename(columns={
    '종목': 'sports_type',
    '구장명': 'stadium_name',
    '지역': 'region',
    '도로명 주소': 'address',
    '지역코드': 'stadium_code'
}, inplace=True)

df_stadium['stadium_code'] = df_stadium['stadium_name'].map(stadium_code_map)

# stadium_code가 모두 추가되었는지 확인
missing_codes = df_stadium[df_stadium['stadium_code'].isnull()]
print(f"stadium_code가 설정되지 않은 경기장 수: {len(missing_codes)}")

# 매핑되지 않은 경기장 이름 목록 보기
if not missing_codes.empty:
    print(missing_codes['stadium_name'].unique(), missing_codes['stadium_name'].unique())

stadium_code가 설정되지 않은 경기장 수: 0


In [5]:
# stadium_code 기준으로 중복 제거 (첫 번째 값만 남기고 나머지 제거)
df_stadium = df_stadium.drop_duplicates(subset=['stadium_code'], keep='first')

# 중복 제거된 데이터 DB에 삽입
df_stadium.to_sql(
    name='stadium',
    con=engine,
    if_exists='append',  # 기존 데이터 유지하며 추가
    index=False
)

60

# 2. sports_game 테이블

## 야구

In [6]:
import pandas as pd
from datetime import datetime, timedelta

# 데이터셋 가져오기
df = pd.read_csv("./dataset/야구_kbo_schedule_2023_2024_transformed.csv")

# DB에 알맞게 컬럼명 수정 및 시간, 날짜 포맷 수정
# df['end_time'] = df['start_time']
df.columns = ['game_date', 'day_of_week', 'start_time', 'stadium_name', 'home_team', 'home_score',
              'away_team', 'away_score', 'home_team_win', 'match_type']

df['sports_type'] = '야구'
df['game_date'] = pd.to_datetime(df['game_date'], format="%Y.%m.%d").dt.date
df['start_time'] = pd.to_datetime(df['start_time'], format="%H:%M").dt.time

# 야구 종료 시간 : start_time + 5시간 (3-6시간)
def add_times(t):
    dt = datetime.combine(datetime.today(), t)
    return (dt + timedelta(hours=3)).time()
df['end_time'] = df['start_time'].apply(add_times)

df.head()

,game_date,day_of_week,start_time,stadium_name,home_team,home_score,away_team,away_score,home_team_win,match_type,sports_type,end_time
0,2023-01-01,일,14:00:00,고양 소노 아레나,고양 캐롯,77,수원 KT,90,False,정규시즌,야구,17:00:00
1,2023-01-01,일,14:00:00,잠실실내체육관,서울 삼성,67,서울 SK,86,False,정규시즌,야구,17:00:00
2,2023-01-01,일,16:00:00,창원체육관,창원 LG,73,울산 현대모비스,77,False,정규시즌,야구,19:00:00
3,2023-01-02,월,19:00:00,원주종합체육관,원주 DB,70,안양 정관장,86,False,정규시즌,야구,22:00:00
4,2023-01-03,화,19:00:00,군산월명체육관,부산 KCC,79,고양 캐롯,72,True,정규시즌,야구,22:00:00


In [7]:
df['stadium_code'] = df['stadium_name'].map(stadium_code_map)

# stadium_code가 모두 추가되었는지 확인
missing_codes = df[df['stadium_code'].isnull()]
print(f"stadium_code가 설정되지 않은 경기장 수: {len(missing_codes)}")

# 매핑되지 않은 경기장 이름 목록 보기
# if not missing_codes.empty:
#     print(missing_codes['stadium_name'].unique())

stadium_code가 설정되지 않은 경기장 수: 0


In [8]:
df.drop(columns=['stadium_name'], inplace=True)
df.head()

,game_date,day_of_week,start_time,home_team,home_score,away_team,away_score,home_team_win,match_type,sports_type,end_time,stadium_code
0,2023-01-01,일,14:00:00,고양 캐롯,77,수원 KT,90,False,정규시즌,야구,17:00:00,GY01
1,2023-01-01,일,14:00:00,서울 삼성,67,서울 SK,86,False,정규시즌,야구,17:00:00,SO06
2,2023-01-01,일,16:00:00,창원 LG,73,울산 현대모비스,77,False,정규시즌,야구,19:00:00,CW02
3,2023-01-02,월,19:00:00,원주 DB,70,안양 정관장,86,False,정규시즌,야구,22:00:00,WJ01
4,2023-01-03,화,19:00:00,부산 KCC,79,고양 캐롯,72,True,정규시즌,야구,22:00:00,GS01


In [9]:
columns_to_insert = [
    "stadium_code", "sports_type", "game_date", "day_of_week",
    "start_time", "end_time", "home_team_win", "match_type"
]

df[columns_to_insert].to_sql(
    name='sports_game',
    con=engine,
    if_exists='append',  # append: 기존 데이터 유지하고 추가
    index=False,         # DataFrame의 인덱스는 사용하지 않음
    dtype={
        "stadium_code": sqlalchemy.String(10),
        "sports_type": sqlalchemy.String(10),
        "game_date": sqlalchemy.Date,
        "day_of_week": sqlalchemy.String(10),
        "start_time": sqlalchemy.Time,
        "end_time": sqlalchemy.Time,
        "home_team_win": sqlalchemy.Boolean,
        "match_type": sqlalchemy.String(50),
    }
)

# is_holiday, audience 누락됨

568

## 축구

In [10]:
import pandas as pd
from datetime import datetime, timedelta

# 데이터셋 가져오기
df = pd.read_csv("./dataset/축구_kl_schedule_2023_2024_transformed.csv")

# DB에 알맞게 컬럼명 수정 및 시간, 날짜 포맷 수정
# df['end_time'] = df['start_time']
df.columns = ['game_date', 'day_of_week', 'start_time', 'stadium_name', 'home_team', 'home_score',
              'away_team', 'away_score', 'home_team_win', 'match_type']

df['sports_type'] = '축구'
df['game_date'] = pd.to_datetime(df['game_date'], format="%Y.%m.%d").dt.date
df['start_time'] = pd.to_datetime(df['start_time'], format="%H:%M").dt.time

# 축구 종료 시간 : start_time + 2시간
def add_times(t):
    dt = datetime.combine(datetime.today(), t)
    return (dt + timedelta(hours=2)).time()
df['end_time'] = df['start_time'].apply(add_times)

df.head()

,game_date,day_of_week,start_time,stadium_name,home_team,home_score,away_team,away_score,home_team_win,match_type,sports_type,end_time
0,2023-02-25,토,14:00:00,울산 문수 축구경기장,울산,2,전북,1,True,1R,축구,16:00:00
1,2023-02-25,토,16:30:00,서울 월드컵 경기장,서울,2,인천,1,True,1R,축구,18:30:00
2,2023-02-25,토,16:30:00,수원 월드컵 경기장,수원,0,광주,1,False,1R,축구,18:30:00
3,2023-02-26,일,14:00:00,제주 월드컵 경기장,제주,0,수원FC,0,False,1R,축구,16:00:00
4,2023-02-26,일,14:30:00,포항 스틸야드,포항,3,대구,2,True,1R,축구,16:30:00


In [11]:
df['stadium_code'] = df['stadium_name'].map(stadium_code_map)

# stadium_code가 모두 추가되었는지 확인
missing_codes = df[df['stadium_code'].isnull()]
print(f"stadium_code가 설정되지 않은 경기장 수: {len(missing_codes)}")

#매핑되지 않은 경기장 이름 목록 보기
if not missing_codes.empty:
    print(missing_codes['stadium_name'].unique())

stadium_code가 설정되지 않은 경기장 수: 0


In [12]:
df.drop(columns=['stadium_name'], inplace=True)
df.head()

,game_date,day_of_week,start_time,home_team,home_score,away_team,away_score,home_team_win,match_type,sports_type,end_time,stadium_code
0,2023-02-25,토,14:00:00,울산,2,전북,1,True,1R,축구,16:00:00,US01
1,2023-02-25,토,16:30:00,서울,2,인천,1,True,1R,축구,18:30:00,SO01
2,2023-02-25,토,16:30:00,수원,0,광주,1,False,1R,축구,18:30:00,SU01
3,2023-02-26,일,14:00:00,제주,0,수원FC,0,False,1R,축구,16:00:00,JE01
4,2023-02-26,일,14:30:00,포항,3,대구,2,True,1R,축구,16:30:00,PH01


In [13]:
columns_to_insert = [
    "stadium_code", "sports_type", "game_date", "day_of_week",
    "start_time", "end_time", "home_team_win", "match_type"
]

df[columns_to_insert].to_sql(
    name='sports_game',
    con=engine,
    if_exists='append',  # append: 기존 데이터 유지하고 추가
    index=False,         # DataFrame의 인덱스는 사용하지 않음
    dtype={
        "stadium_code": sqlalchemy.String(10),
        "sports_type": sqlalchemy.String(10),
        "game_date": sqlalchemy.Date,
        "day_of_week": sqlalchemy.String(10),
        "start_time": sqlalchemy.Time,
        "end_time": sqlalchemy.Time,
        "home_team_win": sqlalchemy.Boolean,
        "match_type": sqlalchemy.String(50),
    }
)

# is_holiday, audience 누락됨

464

## 농구

In [14]:
import pandas as pd
from datetime import datetime, timedelta

# 데이터셋 가져오기
df = pd.read_csv("./dataset/농구_kbl_schedule_2023_2024_transformed.csv")

# DB에 알맞게 컬럼명 수정 및 시간, 날짜 포맷 수정
# df['end_time'] = df['start_time']
df.columns = ['game_date', 'day_of_week', 'start_time', 'stadium_name', 'home_team', 'home_score',
              'away_team', 'away_score', 'home_team_win', 'match_type']

df['sports_type'] = '농구'
df['game_date'] = pd.to_datetime(df['game_date'], format="%Y.%m.%d").dt.date
df['start_time'] = pd.to_datetime(df['start_time'], format="%H:%M").dt.time

# 농구 종료 시간 : start_time + 80분
def add_times(t):
    dt = datetime.combine(datetime.today(), t)
    return (dt + timedelta(minutes=80)).time()
df['end_time'] = df['start_time'].apply(add_times)

df.head()

,game_date,day_of_week,start_time,stadium_name,home_team,home_score,away_team,away_score,home_team_win,match_type,sports_type,end_time
0,2023-01-01,일,14:00:00,고양 소노 아레나,고양 캐롯,77,수원 KT,90,False,정규시즌,농구,15:20:00
1,2023-01-01,일,14:00:00,잠실실내체육관,서울 삼성,67,서울 SK,86,False,정규시즌,농구,15:20:00
2,2023-01-01,일,16:00:00,창원체육관,창원 LG,73,울산 현대모비스,77,False,정규시즌,농구,17:20:00
3,2023-01-02,월,19:00:00,원주종합체육관,원주 DB,70,안양 정관장,86,False,정규시즌,농구,20:20:00
4,2023-01-03,화,19:00:00,군산월명체육관,부산 KCC,79,고양 캐롯,72,True,정규시즌,농구,20:20:00


In [15]:
df['stadium_code'] = df['stadium_name'].map(stadium_code_map)

# stadium_code가 모두 추가되었는지 확인
missing_codes = df[df['stadium_code'].isnull()]
print(f"stadium_code가 설정되지 않은 경기장 수: {len(missing_codes)}")

#매핑되지 않은 경기장 이름 목록 보기
if not missing_codes.empty:
    print(missing_codes['stadium_name'].unique())

stadium_code가 설정되지 않은 경기장 수: 0


In [16]:
df.drop(columns=['stadium_name'], inplace=True)
df.head()

,game_date,day_of_week,start_time,home_team,home_score,away_team,away_score,home_team_win,match_type,sports_type,end_time,stadium_code
0,2023-01-01,일,14:00:00,고양 캐롯,77,수원 KT,90,False,정규시즌,농구,15:20:00,GY01
1,2023-01-01,일,14:00:00,서울 삼성,67,서울 SK,86,False,정규시즌,농구,15:20:00,SO06
2,2023-01-01,일,16:00:00,창원 LG,73,울산 현대모비스,77,False,정규시즌,농구,17:20:00,CW02
3,2023-01-02,월,19:00:00,원주 DB,70,안양 정관장,86,False,정규시즌,농구,20:20:00,WJ01
4,2023-01-03,화,19:00:00,부산 KCC,79,고양 캐롯,72,True,정규시즌,농구,20:20:00,GS01


In [17]:
columns_to_insert = [
    "stadium_code", "sports_type", "game_date", "day_of_week",
    "start_time", "end_time", "home_team_win", "match_type"
]

df[columns_to_insert].to_sql(
    name='sports_game',
    con=engine,
    if_exists='append',  # append: 기존 데이터 유지하고 추가
    index=False,         # DataFrame의 인덱스는 사용하지 않음
    dtype={
        "stadium_code": sqlalchemy.String(10),
        "sports_type": sqlalchemy.String(10),
        "game_date": sqlalchemy.Date,
        "day_of_week": sqlalchemy.String(10),
        "start_time": sqlalchemy.Time,
        "end_time": sqlalchemy.Time,
        "home_team_win": sqlalchemy.Boolean,
        "match_type": sqlalchemy.String(50),
    }
)

# is_holiday, audience 누락됨

568

## 배구(vl)

In [18]:
import pandas as pd
from datetime import datetime, timedelta

# 데이터셋 가져오기
df = pd.read_csv("./dataset/배구_vl_schedule_2023_2024_transformed.csv")

# DB에 알맞게 컬럼명 수정 및 시간, 날짜 포맷 수정
# df['end_time'] = df['start_time']
df.columns = ['game_date', 'day_of_week', 'start_time', 'stadium_name', 'home_team', 'home_score',
              'away_team', 'away_score', 'home_team_win', 'match_type']

df['sports_type'] = '배구'
df['game_date'] = pd.to_datetime(df['game_date'], format="%Y.%m.%d").dt.date
df['start_time'] = pd.to_datetime(df['start_time'], format="%H:%M").dt.time

# 배구 종료 시간 : start_time + 2시간
def add_times(t):
    dt = datetime.combine(datetime.today(), t)
    return (dt + timedelta(hours=2)).time()
df['end_time'] = df['start_time'].apply(add_times)

df.head()

,game_date,day_of_week,start_time,stadium_name,home_team,home_score,away_team,away_score,home_team_win,match_type,sports_type,end_time
0,2023-01-01,일,14:00:00,안산상록수체육관,OK저축은행,3,대한항공,0,True,정규시즌,배구,16:00:00
1,2023-01-03,화,19:00:00,서울장충체육관,우리카드,3,KB손해보험,0,True,정규시즌,배구,21:00:00
2,2023-01-04,수,19:00:00,인천계양체육관,대한항공,3,OK저축은행,0,True,정규시즌,배구,21:00:00
3,2023-01-05,목,19:00:00,대전충무체육관,삼성화재,3,한국전력,2,True,정규시즌,배구,21:00:00
4,2023-01-06,금,19:00:00,의정부체육관,KB손해보험,3,우리카드,0,True,정규시즌,배구,21:00:00


In [19]:
df['stadium_code'] = df['stadium_name'].map(stadium_code_map)

# stadium_code가 모두 추가되었는지 확인
missing_codes = df[df['stadium_code'].isnull()]
print(f"stadium_code가 설정되지 않은 경기장 수: {len(missing_codes)}")

#매핑되지 않은 경기장 이름 목록 보기
if not missing_codes.empty:
    print(missing_codes['stadium_name'].unique())

stadium_code가 설정되지 않은 경기장 수: 0


In [20]:
df.drop(columns=['stadium_name'], inplace=True)
df.head()

,game_date,day_of_week,start_time,home_team,home_score,away_team,away_score,home_team_win,match_type,sports_type,end_time,stadium_code
0,2023-01-01,일,14:00:00,OK저축은행,3,대한항공,0,True,정규시즌,배구,16:00:00,AS01
1,2023-01-03,화,19:00:00,우리카드,3,KB손해보험,0,True,정규시즌,배구,21:00:00,SO08
2,2023-01-04,수,19:00:00,대한항공,3,OK저축은행,0,True,정규시즌,배구,21:00:00,IC03
3,2023-01-05,목,19:00:00,삼성화재,3,한국전력,2,True,정규시즌,배구,21:00:00,DJ04
4,2023-01-06,금,19:00:00,KB손해보험,3,우리카드,0,True,정규시즌,배구,21:00:00,UJ01


In [21]:
columns_to_insert = [
    "stadium_code", "sports_type", "game_date", "day_of_week",
    "start_time", "end_time", "home_team_win", "match_type"
]

df[columns_to_insert].to_sql(
    name='sports_game',
    con=engine,
    if_exists='append',  # append: 기존 데이터 유지하고 추가
    index=False,         # DataFrame의 인덱스는 사용하지 않음
    dtype={
        "stadium_code": sqlalchemy.String(10),
        "sports_type": sqlalchemy.String(10),
        "game_date": sqlalchemy.Date,
        "day_of_week": sqlalchemy.String(10),
        "start_time": sqlalchemy.Time,
        "end_time": sqlalchemy.Time,
        "home_team_win": sqlalchemy.Boolean,
        "match_type": sqlalchemy.String(50),
    }
)

# is_holiday, audience 누락됨

282

## 배구(여자배구, wvl)

In [22]:
import pandas as pd
from datetime import datetime, timedelta

# 데이터셋 가져오기
df = pd.read_csv("./dataset/배구_여자_wvl_schedule_2023_2024_transformed.csv")

# DB에 알맞게 컬럼명 수정 및 시간, 날짜 포맷 수정
# df['end_time'] = df['start_time']
df.columns = ['game_date', 'day_of_week', 'start_time', 'stadium_name', 'home_team', 'home_score',
              'away_team', 'away_score', 'home_team_win', 'match_type']

df['sports_type'] = '여자배구'
df['game_date'] = pd.to_datetime(df['game_date'], format="%Y.%m.%d").dt.date
df['start_time'] = pd.to_datetime(df['start_time'], format="%H:%M").dt.time

# 배구 종료 시간 : start_time + 2시간
def add_times(t):
    dt = datetime.combine(datetime.today(), t)
    return (dt + timedelta(hours=2)).time()
df['end_time'] = df['start_time'].apply(add_times)

df.head()

,game_date,day_of_week,start_time,stadium_name,home_team,home_score,away_team,away_score,home_team_win,match_type,sports_type,end_time
0,2023-01-01,일,16:00:00,화성종합실내체육관,IBK기업은행,0,현대건설,3,False,정규시즌,여자배구,18:00:00
1,2023-01-03,화,19:00:00,대전충무체육관,정관장,2,한국도로공사,3,False,정규시즌,여자배구,21:00:00
2,2023-01-04,수,19:00:00,수원체육관,현대건설,3,IBK기업은행,0,True,정규시즌,여자배구,21:00:00
3,2023-01-05,목,19:00:00,인천삼산월드체육관,흥국생명,3,GS칼텍스,2,True,정규시즌,여자배구,21:00:00
4,2023-01-06,금,19:00:00,김천실내체육관,한국도로공사,3,정관장,1,True,정규시즌,여자배구,21:00:00


In [23]:
df['stadium_code'] = df['stadium_name'].map(stadium_code_map)

# stadium_code가 모두 추가되었는지 확인
missing_codes = df[df['stadium_code'].isnull()]
print(f"stadium_code가 설정되지 않은 경기장 수: {len(missing_codes)}")

#매핑되지 않은 경기장 이름 목록 보기
if not missing_codes.empty:
    print(missing_codes['stadium_name'].unique())

stadium_code가 설정되지 않은 경기장 수: 0


In [24]:
df.drop(columns=['stadium_name'], inplace=True)
df.head()

,game_date,day_of_week,start_time,home_team,home_score,away_team,away_score,home_team_win,match_type,sports_type,end_time,stadium_code
0,2023-01-01,일,16:00:00,IBK기업은행,0,현대건설,3,False,정규시즌,여자배구,18:00:00,HS02
1,2023-01-03,화,19:00:00,정관장,2,한국도로공사,3,False,정규시즌,여자배구,21:00:00,DJ04
2,2023-01-04,수,19:00:00,현대건설,3,IBK기업은행,0,True,정규시즌,여자배구,21:00:00,SU05
3,2023-01-05,목,19:00:00,흥국생명,3,GS칼텍스,2,True,정규시즌,여자배구,21:00:00,IC04
4,2023-01-06,금,19:00:00,한국도로공사,3,정관장,1,True,정규시즌,여자배구,21:00:00,GC02


In [25]:
columns_to_insert = [
    "stadium_code", "sports_type", "game_date", "day_of_week",
    "start_time", "end_time", "home_team_win", "match_type"
]

df[columns_to_insert].to_sql(
    name='sports_game',
    con=engine,
    if_exists='append',  # append: 기존 데이터 유지하고 추가
    index=False,         # DataFrame의 인덱스는 사용하지 않음
    dtype={
        "stadium_code": sqlalchemy.String(10),
        "sports_type": sqlalchemy.String(10),
        "game_date": sqlalchemy.Date,
        "day_of_week": sqlalchemy.String(10),
        "start_time": sqlalchemy.Time,
        "end_time": sqlalchemy.Time,
        "home_team_win": sqlalchemy.Boolean,
        "match_type": sqlalchemy.String(50),
    }
)

# is_holiday, audience 누락됨

282

# 3. weather 테이블

In [26]:
import pandas as pd
from datetime import datetime, timedelta

# 데이터셋 가져오기
df_2023 = pd.read_csv("./dataset/2023년_기상관측정보(최종).csv", sep='\t', encoding='utf-16')
df_2024 = pd.read_csv("./dataset/2024년_기상관측정보(최종).csv", sep='\t', encoding='utf-16')

df_merged = pd.concat([df_2023, df_2024], join='inner', ignore_index=True)
df_merged = df_merged.drop(columns=['지점'])
df_merged = df_merged.drop(columns=['일조(hr)','일조 QC플래그','일사(MJ/m2)','일사 QC플래그'])

# QC플래그에 대한 전처리 필요
df_merged = df_merged.drop(columns=['기온 QC플래그','강수량 QC플래그','풍속 QC플래그','풍향 QC플래그','습도 QC플래그'])

In [27]:
# 컬럼명 수정
df_merged = df_merged.rename(columns={
    "지점명": "region",
    "기온(°C)": "temperature",
    "강수량(mm)": "precipitation",
    "풍속(m/s)": "wind_speed",
    "풍향(16방위)": "wind_dir_deg",
    "습도(%)": "humidity",
    "적설(cm)": "snow_depth",
    "전운량(10분위)": "cloud_amount",
    "중하층운량(10분위)": "low_cloud_amt",
    "시정(10m)": "visibility",
})

In [28]:
df_merged.head()

,region,일시,temperature,precipitation,wind_speed,wind_dir_deg,humidity,snow_depth,cloud_amount,low_cloud_amt,visibility
0,춘천,2023-01-01 0:00,-2.5,NaN,0.6,340.0,88.0,1.4,9.0,NaN,630.0
1,춘천,2023-01-01 1:00,-2.8,NaN,0.5,360.0,89.0,1.4,0.0,NaN,561.0
2,춘천,2023-01-01 2:00,-3.0,NaN,0.4,0.0,89.0,1.5,0.0,NaN,598.0
3,춘천,2023-01-01 3:00,-3.2,NaN,0.2,0.0,90.0,1.5,0.0,NaN,513.0
4,춘천,2023-01-01 4:00,-4.7,NaN,0.5,360.0,93.0,1.5,0.0,NaN,268.0


In [29]:
# 일시(2023-01-01 0:00) -> 날짜, 시간 정보 따로 저장
df_merged['일시'] = pd.to_datetime(df_merged['일시'], format='%Y-%m-%d %H:%M')
df_merged['weather_date'] = df_merged['일시'].dt.date
df_merged['weather_time'] = df_merged['일시'].dt.time
df_merged = df_merged.drop(columns=['일시'])  # 기존 컬럼 삭제

In [30]:
# stadium_code 설정
stadium_code_dict = {
    "안산": ["AS01"],
    "안양": ["AY01"],
    "부산": ["BS01", "BS02", "BS03"],
    "천안": ["CA01", "CA02"],
    "춘천": ["CC01"],
    "청주": ["CJ01"],
    "창원": ["CW01", "CW02"],
    "대구": ["DG01", "DG02", "DG03", "DG04"],
    "대전": ["DJ01", "DJ02", "DJ03", "DJ04"],
    "김천": ["GC01", "GC02"],
    "광주": ["GJ01", "GJ02"],
    "구미": ["GM01"],
    "강릉": ["GN01"],
    "김포": ["GP01"],
    "군산": ["GS01"],
    "고양": ["GY01"],
    "화성": ["HS01", "HS02"],
    "인천": ["IC01", "IC02", "IC03", "IC04"],
    "이천": ["IH01"],
    "제주": ["JE01"],
    "전주": ["JJ01", "JJ02"],
    "포항": ["PH01", "PH02"],
    "서울": ["SO01", "SO02", "SO03", "SO04", "SO05", "SO06", "SO07", "SO08"],
    "수원": ["SU01", "SU02", "SU03", "SU04", "SU05"],
    "의정부": ["UJ01", "UJ02"],
    "울산": ["US01", "US02", "US03", "US04"],
    "원주": ["WJ01"]
}

# '지점명' 컬럼을 기준으로 stadium_code_dict에서 첫 번째 stadium_code 저장
df_merged['stadium_code'] = df_merged['region'].map(lambda x: stadium_code_dict.get(x, [None])[0])

# 매핑되지 않은 데이터(None) 확인
none_count = df_merged['stadium_code'].isnull().sum()
print(f"매핑되지 않은 데이터 수: {none_count}")

if none_count > 0:
    print(df_merged[df_merged['stadium_code'].isnull()][['region', 'stadium_code']])

매핑되지 않은 데이터 수: 0


In [31]:
df_merged.head()

,region,temperature,precipitation,wind_speed,wind_dir_deg,humidity,snow_depth,cloud_amount,low_cloud_amt,visibility,weather_date,weather_time,stadium_code
0,춘천,-2.5,NaN,0.6,340.0,88.0,1.4,9.0,NaN,630.0,2023-01-01,00:00:00,CC01
1,춘천,-2.8,NaN,0.5,360.0,89.0,1.4,0.0,NaN,561.0,2023-01-01,01:00:00,CC01
2,춘천,-3.0,NaN,0.4,0.0,89.0,1.5,0.0,NaN,598.0,2023-01-01,02:00:00,CC01
3,춘천,-3.2,NaN,0.2,0.0,90.0,1.5,0.0,NaN,513.0,2023-01-01,03:00:00,CC01
4,춘천,-4.7,NaN,0.5,360.0,93.0,1.5,0.0,NaN,268.0,2023-01-01,04:00:00,CC01


In [32]:
columns_to_insert = [
    "stadium_code", "region", "weather_date", "weather_time",
    "temperature", "precipitation", "wind_speed", "wind_dir_deg",
    "humidity", "snow_depth", "cloud_amount", "low_cloud_amt", "visibility"
]

df_merged[columns_to_insert].to_sql(
    name='weather',
    con=engine,
    if_exists='append',  # 기존 데이터 유지하고 추가
    index=False,
    dtype={
        "stadium_code": sqlalchemy.String(10),
        "region": sqlalchemy.String(20),
        "weather_date": sqlalchemy.Date,
        "weather_time": sqlalchemy.Time,
        "temperature": sqlalchemy.Float,
        "precipitation": sqlalchemy.Float,
        "wind_speed": sqlalchemy.Float,
        "wind_dir_deg": sqlalchemy.Integer,
        "humidity": sqlalchemy.Float,
        "snow_depth": sqlalchemy.Float,
        "cloud_amount": sqlalchemy.Integer,
        "low_cloud_amt": sqlalchemy.Integer,
        "visibility": sqlalchemy.Integer,
    }
)


450811

In [33]:
# # 외래키 오류 발생 시 디버깅
# ## IntegrityError: (1452, 'Cannot add or update a child row: a foreign key constraint fails (aiclass2.weather, CONSTRAINT weather_ibfk_1 FOREIGN KEY (stadium_code) REFERENCES stadium (stadium_code))')

# # 1. stadium 테이블의 stadium_code 목록 불러오기
# stadium_codes_in_db = pd.read_sql("SELECT stadium_code FROM stadium", con=engine)

# # 2. df_merged의 stadium_code 중 DB에 없는 값 찾기
# merged_codes = df_merged['stadium_code'].unique()
# db_codes = stadium_codes_in_db['stadium_code'].unique()

# missing_codes = set(merged_codes) - set(db_codes)
# print("DB에 없는 stadium_code:", missing_codes)

# 4. traffic_accident 테이블

In [34]:
import pandas as pd
df_2023_accident = pd.read_csv("./dataset/2023년_스포츠경기장주변_교통사고_transformed.csv")
df_2024_accident = pd.read_csv("./dataset/2024년_스포츠경기장주변_교통사고_transformed.csv")

df_accident_merged = pd.concat([df_2023_accident, df_2024_accident], join='inner', ignore_index=True)
df_accident_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34032 entries, 0 to 34031
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      34032 non-null  object
 1   날짜      34032 non-null  object
 2   사고      34032 non-null  int64 
 3   사망      34032 non-null  int64 
 4   부상      34032 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.3+ MB


In [35]:
df_accident_merged = df_accident_merged.rename(columns={
    "지역": "region",
    "날짜": "accident_date",
    "사고": "accident_count",
    "사망": "death_count",
    "부상": "injury_count"
})

df_accident_merged.head(5)

,region,accident_date,accident_count,death_count,injury_count
0,강원 강릉시,2023.01.01,5,0,10
1,강원 강릉시,2023.01.02,2,0,2
2,강원 강릉시,2023.01.03,3,0,4
3,강원 강릉시,2023.01.04,3,0,4
4,강원 강릉시,2023.01.05,1,0,1


In [36]:
df_accident_merged['accident_date'] = pd.to_datetime(df_accident_merged['accident_date'], format="%Y.%m.%d")
df_accident_merged.head(5)                                        

,region,accident_date,accident_count,death_count,injury_count
0,강원 강릉시,2023-01-01,5,0,10
1,강원 강릉시,2023-01-02,2,0,2
2,강원 강릉시,2023-01-03,3,0,4
3,강원 강릉시,2023-01-04,3,0,4
4,강원 강릉시,2023-01-05,1,0,1


In [37]:
df_accident_merged['region'].unique()

array(['강원 강릉시', '강원 원주시', '강원 춘천시', '경기 수원시', '경기 안산시', '경기 의정부시',
       '경기 이천시', '경기 화성시', '경북 구미시', '경북 김천시', '경북 포항시', '광주 광산구',
       '광주 남구', '광주 동구', '광주 북구', '광주 서구', '대구 남구', '대구 동구', '대구 북구',
       '대구 서구', '대구 수성구', '대구 중구', '대전 동구', '대전 서구', '대전 유성구', '대전 중구',
       '부산 남구', '부산 동구', '부산 동래구', '부산 북구', '부산 서구', '부산 연제구', '부산 중구',
       '서울 마포구', '서울 송파구', '서울 양천구', '서울 중구', '울산 남구', '울산 동구', '울산 북구',
       '울산 중구', '인천 계양구', '인천 남동구', '인천 동구', '인천 미추홀구', '인천 서구', '인천 중구',
       '전북 군산시', '전북 전주시', '제주 제주시', '충남 천안시', '충북 청주시'], dtype=object)

In [38]:
# stadium_code 설정
stadium_code_dict_detail = {
    "경기 안산시": ["AS01"],
    "안양": ["AY01"],
    "부산 남구": ["BS01", "BS02", "BS03"],
    "부산 동구": ["BS01", "BS02", "BS03"],
    "부산 동래구": ["BS01", "BS02", "BS03"],
    "부산 북구": ["BS01", "BS02", "BS03"],
    "부산 서구": ["BS01", "BS02", "BS03"],
    "부산 연제구": ["BS01", "BS02", "BS03"],
    "부산 중구": ["BS01", "BS02", "BS03"],
    "충남 천안시": ["CA01", "CA02"],
    "강원 춘천시": ["CC01"],
    "충북 청주시": ["CJ01"],
    "창원": ["CW01", "CW02"],
    "대구 남구": ["DG01", "DG02", "DG03", "DG04"],
    "대구 동구": ["DG01", "DG02", "DG03", "DG04"],
    "대구 북구": ["DG01", "DG02", "DG03", "DG04"],
    "대구 서구": ["DG01", "DG02", "DG03", "DG04"],
    "대구 수성구": ["DG01", "DG02", "DG03", "DG04"],
    "대구 중구": ["DG01", "DG02", "DG03", "DG04"],
    "대전 동구": ["DJ01", "DJ02", "DJ03", "DJ04"],
    "대전 서구": ["DJ01", "DJ02", "DJ03", "DJ04"],
    "대전 유성구": ["DJ01", "DJ02", "DJ03", "DJ04"],
    "대전 중구": ["DJ01", "DJ02", "DJ03", "DJ04"],
    "경북 김천시": ["GC01", "GC02"],
    "광주 광산구": ["GJ01", "GJ02"],
    "광주 남구": ["GJ01", "GJ02"],
    "광주 동구": ["GJ01", "GJ02"],
    "광주 북구": ["GJ01", "GJ02"],
    "광주 서구": ["GJ01", "GJ02"],
    "경북 구미시": ["GM01"],
    "강원 강릉시": ["GN01"],
    "김포": ["GP01"],
    "전북 군산시": ["GS01"],
    "고양": ["GY01"],
    "경기 화성시": ["HS01", "HS02"],
    "인천 계양구": ["IC01", "IC02", "IC03", "IC04"],
    "인천 남동구": ["IC01", "IC02", "IC03", "IC04"],
    "인천 동구": ["IC01", "IC02", "IC03", "IC04"],
    "인천 미추홀구": ["IC01", "IC02", "IC03", "IC04"],
    "인천 서구": ["IC01", "IC02", "IC03", "IC04"],
    "인천 중구": ["IC01", "IC02", "IC03", "IC04"],
    "경기 이천시": ["IH01"],
    "제주 제주시": ["JE01"],
    "전북 전주시": ["JJ01", "JJ02"],
    "경북 포항시": ["PH01", "PH02"],
    "서울 마포구": ["SO01", "SO02", "SO03", "SO04", "SO05", "SO06", "SO07", "SO08"],
    "서울 송파구": ["SO01", "SO02", "SO03", "SO04", "SO05", "SO06", "SO07", "SO08"],
    "서울 양천구": ["SO01", "SO02", "SO03", "SO04", "SO05", "SO06", "SO07", "SO08"],
    "서울 중구": ["SO01", "SO02", "SO03", "SO04", "SO05", "SO06", "SO07", "SO08"],
    "경기 수원시": ["SU01", "SU02", "SU03", "SU04", "SU05"],
    "경기 의정부시": ["UJ01", "UJ02"],
    "울산 남구": ["US01", "US02", "US03", "US04"],
    "울산 동구": ["US01", "US02", "US03", "US04"],
    "울산 북구": ["US01", "US02", "US03", "US04"],
    "울산 중구": ["US01", "US02", "US03", "US04"],
    "강원 원주시": ["WJ01"]
}
# '지점명' 컬럼을 기준으로 stadium_code_dict_detail에서 첫 번째 stadium_code 저장
df_accident_merged['stadium_code'] = df_accident_merged['region'].map(lambda x: stadium_code_dict_detail.get(x, [None])[0])

# 매핑되지 않은 데이터(None) 확인
none_count = df_accident_merged['stadium_code'].isnull().sum()
print(f"매핑되지 않은 데이터 수: {none_count}")

if none_count > 0:
    print(df_accident_merged[df_accident_merged['stadium_code'].isnull()][['region', 'stadium_code']])

매핑되지 않은 데이터 수: 0


In [39]:
columns_to_insert = ["region", "accident_date", "accident_count", "death_count", "injury_count", "stadium_code"]

df_accident_merged[columns_to_insert].to_sql(
    name='traffic_accident',
    con=engine,
    if_exists='append',  # append: 기존 데이터 유지하고 추가
    index=False,         # DataFrame의 인덱스는 사용하지 않음
    dtype={
        "region": sqlalchemy.String(20),
        "accident_date": sqlalchemy.Date,
        "accident_count": sqlalchemy.Integer,
        "death_count": sqlalchemy.Integer,
        "injury_count": sqlalchemy.Integer,
        "stadium_code": sqlalchemy.String(10)
    }
)

34032

# 5. fire_info 테이블

In [45]:
import pandas as pd
from datetime import datetime

# 데이터셋 가져오기
df = pd.read_csv("./dataset/2023~2024_전국_교통사고_소방출동_데이터_20250530_2.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4994 entries, 0 to 4993
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               4994 non-null   int64  
 1   reptDt             4994 non-null   float64
 2   reptAreaNm         4991 non-null   object 
 3   dstrAreaNm         4994 non-null   object 
 4   pttnNm2            4994 non-null   object 
 5   queried_area_code  4994 non-null   int64  
 6   queried_from_date  4994 non-null   int64  
 7   queried_to_date    4994 non-null   int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 312.3+ KB


In [46]:
df = df[['date', 'reptAreaNm', 'queried_area_code']]

# 컬럼명 변경
df.rename(columns={
    'reptAreaNm': 'address',
    'queried_area_code': 'region_code'
}, inplace=True)

# date(연월일시분초) -> fire_date(연월일), fire_time(시분초)로 구분하여 저장
df['datetime'] = pd.to_datetime(df['date'].astype(str), format='%Y%m%d%H%M%S')
df['fire_date'] = df['datetime'].dt.date
df['fire_time'] = df['datetime'].dt.time

df = df.drop(columns=['date', 'datetime'])
df.head()

,address,region_code,fire_date,fire_time
0,서울특별시 종로구 종로6가,11110,2023-01-04,14:38:12
1,서울특별시 종로구 종로6가,11110,2023-01-03,10:24:12
2,서울특별시 종로구 평창동,11110,2023-03-19,15:49:25
3,서울특별시 중구 충무로2가,11140,2023-01-06,10:58:41
4,서울특별시 중구 봉래동2가,11140,2023-04-30,15:52:49


In [47]:
# region : region_code 실제 지역으로 매핑
# traffic_accident 테이블과 호환되는 지역명으로 전처리

df_region = pd.read_csv("./dataset/지역코드.csv")

replacements = {
    '서울특별시': '서울',
    '부산광역시': '부산', '부산직할시': '부산',
    '대구광역시': '대구', '대구직할시': '대구', 
    '인천광역시': '인천', '인천직할시': '인천',
    '광주광역시': '광주', '광주직할시': '광주',
    '대전광역시': '대전', '대전직할시': '대전',
    '울산광역시': '울산',
    '세종특별자치시': '세종',
    '제주도': '제주', '제주특별자치도': '제주',
    '경기도': '경기',
    '강원도': '강원', '강원특별자치도': '강원',
    '충청북도': '충북',
    '충청남도': '충남',
    '전라북도': '전북', '전북특별자치도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남'
}


def normalize_address(addr):
    for full, short in replacements.items():
        if addr.startswith(full):
            addr_trimmed = addr.replace(full, short, 1).strip()
            parts = addr_trimmed.split()

            if full.endswith(('특별시', '광역시', '직할시', '특별자치시')):
                return ' '.join(parts)

            # 도 단위는 도 + 시/군까지만 사용 (3단어 이상이면 뒤는 제거)
            if len(parts) >= 2:
                return ' '.join(parts[:2])
            else:
                return ' '.join(parts)
    return addr.strip()

df_region['address'] = df_region['address'].apply(normalize_address)
df_region.head()

,code,address
0,11110,서울 종로구
1,11140,서울 중구
2,11170,서울 용산구
3,11200,서울 성동구
4,11215,서울 광진구


In [48]:
# df의 region 컬럼을 df_region의 address로 덮어쓰기
code_to_region = df_region.set_index('code')['address']
df['region'] = df['region_code'].map(code_to_region)

df.head()

,address,region_code,fire_date,fire_time,region
0,서울특별시 종로구 종로6가,11110,2023-01-04,14:38:12,서울 종로구
1,서울특별시 종로구 종로6가,11110,2023-01-03,10:24:12,서울 종로구
2,서울특별시 종로구 평창동,11110,2023-03-19,15:49:25,서울 종로구
3,서울특별시 중구 충무로2가,11140,2023-01-06,10:58:41,서울 중구
4,서울특별시 중구 봉래동2가,11140,2023-04-30,15:52:49,서울 중구


In [49]:
df.to_sql(
    name='fire_info',
    con=engine,
    schema='aiclass2',
    if_exists='append',
    index=False,             # DataFrame의 인덱스를 테이블에 포함하지 않음
    dtype={
        'fire_date': sqlalchemy.Date,
        'fire_time': sqlalchemy.Time,
        'address': sqlalchemy.String(200),
        'region': sqlalchemy.String(20),
        'region_code': sqlalchemy.Integer
    }
)

4994